In [ ]:
# Standard library imports
import os
import random
import warnings
import glob

# Third-party imports
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from PIL import Image
from skimage.io import imread
from skimage.transform import resize

# Setting up matplotlib to work interactively in a Jupyter environment
%matplotlib inline

# Setting the seed for reproducibility
seed_value = 42
random.seed(seed_value)  # For Python's built-in random module
np.random.seed(seed_value)  # For NumPy
tf.random.set_seed(seed_value)  # For TensorFlow

# Don't show warning messages
warnings.filterwarnings('ignore')


### Data generator Keras


In [ ]:
def get_mean_and_std(data_path):
    # Calculate the mean and standard deviation of the array of all images
    sum_pixels = 0
    num_pixels = 0

    # Loop over all images in the dataset directory
    mean=[]
    std = []
    for filename in glob.glob(data_path+"\*"):
        # Load the image
        img = cv2.imread( filename,0)/255.

        # Convert the image to a NumPy array
        img_array = np.array(img)

        # Update the running sum of pixel values and the number of pixels
        mean.append(np.mean(img_array))
        std.append(np.std(img_array))
    mean = np.mean(np.array(mean))
    std = np.mean(np.array(std))

    print(f'mean={mean}, std={std}')

In [ ]:
def adjust_brightness_saturation_hue(image, brightness, saturation, hue):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image = hsv_image.astype(np.float32)

    hsv_image[..., 0] = (hsv_image[..., 0] + hue * 360) % 180
    hsv_image[..., 1] = np.clip(hsv_image[..., 1] * saturation, 0, 255)
    hsv_image[..., 2] = np.clip(hsv_image[..., 2] * brightness, 0, 255)

    hsv_image = hsv_image.astype(np.uint8)
    hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2GRAY)
    return hsv_image

In [ ]:
def image_data_generator(data_files, data_dir, lab_dir, batch_size, data_aug=False, main_test=False):
    while True:
        np.random.shuffle(data_files)
        for i in range(0, len(data_files), batch_size):
            batch_files = data_files[i:i + batch_size]
            batch_images = []
            batch_labels = []
            for file in batch_files:
                try:
                    # print(f"Reading file: {file}")  # This line is commented out

                    image = cv2.imread(data_dir + file, 0)
                    if image is None:
                        print(f"Image not found or unable to open: {data_dir + file}")
                        continue

                    if data_aug:
                        if random.getrandbits(1):
                            # Apply brightness, saturation, and hue adjustments
                            image = adjust_brightness_saturation_hue(image, BRIGHTNESS, SATURATION, HUE)

                    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))/255.
                    if per_image_norm:
                        image = (image - np.mean(image))/np.std(image)
                    else:
                        image = (image - mean)/std
                    image = Image.fromarray(image)
                    image = tf.convert_to_tensor(np.array(image))

                    if main_test:
                        mask = cv2.imread(lab_dir + file, cv2.IMREAD_COLOR)
                    else:
                       
                        mask = cv2.imread(lab_dir + file, cv2.IMREAD_COLOR)

                    if mask is None:
                        print(f"Mask not found or unable to open: {lab_dir + file}")
                        continue

                    mask = cv2.resize(mask, (IMG_HEIGHT, IMG_WIDTH))
                    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
                    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) / 255.
                    mask = mask.reshape(IMG_HEIGHT, IMG_WIDTH, 1)

                    batch_images.append(image)
                    batch_labels.append(mask)

                except Exception as e:
                    print(f"Error processing file: {file}. Error: {e}")
                    continue

            batch_images = np.array(batch_images)
            batch_labels = np.array(batch_labels)

            yield batch_images, batch_labels


In [ ]:
def preprocess_and_load(data_path, part_data_path, part_lab_path, lab_path, train_list, test_list):
    val_set = 0.2
    btch_sz =8
    t_list = train_list[0:int((1-val_set)*len(train_list))]
    v_list = train_list[int((1-val_set)*len(train_list)):]
    train_gen = image_data_generator(t_list,part_data_path,part_lab_path, batch_size=btch_sz, data_aug=True, main_test=True)
    val_gen = image_data_generator(v_list,part_data_path,part_lab_path, batch_size=btch_sz, data_aug=False, main_test=True)


    test_gen_large = image_data_generator(test_list,data_path,lab_path, batch_size=btch_sz, data_aug=False, main_test=True)
    return train_gen, val_gen, test_gen_large

# Sample Usage

In [ ]:
# Create empty arrays
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
IMG_CHANNELS = 1
per_image_norm = False

# Add these constants for brightness, saturation, and hue adjustments
BRIGHTNESS = 0.2
SATURATION = 0.3
HUE = 0.25

# Define Paths
data_path = r'< full_train_data_path >' 
lab_path =  r'< full_test_data_path >'
part_data_path = r'< part_train_data >'   
part_lab_path = r'< part_test_data >'
part_train_list = os.listdir(part_data_path)
test_list = os.listdir(data_path)

get_mean_and_std(data_path)
train_gen, val_gen, test_gen_large = preprocess_and_load(data_path, part_data_path, part_lab_path, lab_path, train_list, test_list)